# Create training movies from csv file
In this notebook, we create training movies using the csv files generated by our MouseClick Fiji plugin which writes the event location as t,x,y,boolean format. The user has to input the image path used for creating the csv file along with the csv file. Other parameters to be specified by the user are indicated preceeding their respective blocks.

In [1]:
import numpy as np
from tifffile import imread 
import sys
import os
from glob import glob
sys.path.append("../NEAT")
from  NEATUtils import Augmentation
from NEATUtils import MovieCreator, npzfileGenerator, Upsample
try:
    from pathlib import Path
    Path().expanduser()
except (ImportError,AttributeError):
    from pathlib2 import Path

try:
    import tempfile
    tempfile.TemporaryDirectory
except (ImportError,AttributeError):
    from backports import tempfile


In the block below specify the image used fore creating the csv files along with the total categories for classification. 

The default categories are: Norman events , Apoptosis, Division, Macrocheate, NonMature and Mature P1 cells. 

Crop size specifies the XY crop to be created at the location of the dot in the csv file and the third dimension is the crop in time relative to the click location, so it is X,Y,T, Crops image area sizeX = -X/2:X/2, -Y/2:Y/2, -T:T, so 7 Frame movies if T = 3

Since the network learns the joint task of classification and localization, we need to create that anwser int eh training data itself. Hence user specifies the value of shift in pixels to shift the event to, the program moves the event location from the center to nine locations (left, right, bottom, top, upper-left/right and bottom-left/right)

The task of the program below is to create movie crops and assigned training label to each crop, labels used for events are:

Normal Event: Label 0
Apoptosis Event: Label 1
Division Event: Label 2
Macrocheate Event: Label 3
Non MatureP1 Event : Label 4
Mature Event: Label 5

So for division in the center the training label would be
[0,0,1,0,0,0.5,0.5,0.5]
for the same event on the top Left
[0,0,1,0,0,0.38,0.38,0.5] and so on. This task is done by the program automatically

In [2]:

Localizationsave_dir = '/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neaton/'
LocalizationNormalsave_dir = Localizationsave_dir +  'DynamicNormalEventTrainingDataBin2'
LocalizationApoptosissave_dir = Localizationsave_dir + 'DynamicMasterApoptosisEventTrainingDataBin2'
LocalizationDivisionsave_dir = Localizationsave_dir + 'DynamicMasterDivisionEventTrainingDataBin2'
LocalizationMacrocheatesave_dir = Localizationsave_dir + 'DynamicMacroKittyEventTrainingDataBin2'
LocalizationNonMaturesave_dir = Localizationsave_dir + 'DynamicNonMatureP1EventTrainingDataBin2'
LocalizationMaturesave_dir = Localizationsave_dir +  'DynamicMatureP1EventTrainingDataBin2'

Path(LocalizationNormalsave_dir).mkdir(exist_ok = True)
Path(LocalizationApoptosissave_dir).mkdir(exist_ok = True)
Path(LocalizationDivisionsave_dir).mkdir(exist_ok = True)
Path(LocalizationMacrocheatesave_dir).mkdir(exist_ok = True)
Path(LocalizationNonMaturesave_dir).mkdir(exist_ok = True)
Path(LocalizationMaturesave_dir).mkdir(exist_ok = True)

SaveNpzDirectory = '/home/sancere/VarunNewton/CurieTrainingDatasets/O-NEAT/'

crop_size = [54,54,3]
SizeX = crop_size[0]
SizeY = crop_size[1]

TotalCategories = 6


In [3]:
DirectoryList = []
LabelList = []

Subdir = next(os.walk(LocalizationNormalsave_dir))

for x in Subdir[1]:
    currentdir = LocalizationNormalsave_dir + '/' + x
   
    SubSubdir = next(os.walk(currentdir))
    for y in SubSubdir[1]:
        alldir = LocalizationNormalsave_dir + '/' + x + '/' + y+ '/'
        
        DirectoryList.append(alldir)
      
        
Subdir = next(os.walk(LocalizationApoptosissave_dir))

for x in Subdir[1]:
    currentdir = LocalizationApoptosissave_dir + '/' + x 
    
    SubSubdir = next(os.walk(currentdir))
    for y in SubSubdir[1]:
        alldir = LocalizationApoptosissave_dir + '/' + x + '/' + y+ '/'
        DirectoryList.append(alldir)
        
        
Subdir = next(os.walk(LocalizationDivisionsave_dir))

for x in Subdir[1]:
    currentdir = LocalizationDivisionsave_dir + '/' + x 
    SubSubdir = next(os.walk(currentdir))
    for y in SubSubdir[1]:
        alldir = LocalizationDivisionsave_dir + '/' + x + '/' + y+ '/'
        DirectoryList.append(alldir)
        
Subdir = next(os.walk(LocalizationMacrocheatesave_dir))

for x in Subdir[1]:
    currentdir = LocalizationMacrocheatesave_dir + '/' + x 
    SubSubdir = next(os.walk(currentdir))
    for y in SubSubdir[1]:
        alldir = LocalizationMacrocheatesave_dir + '/' + x + '/' + y+ '/'
        DirectoryList.append(alldir)
        
Subdir = next(os.walk(LocalizationNonMaturesave_dir))

for x in Subdir[1]:
    currentdir = LocalizationNonMaturesave_dir + '/' + x 
    SubSubdir = next(os.walk(currentdir))
    for y in SubSubdir[1]:
        alldir = LocalizationNonMaturesave_dir + '/' + x + '/' + y+ '/'
        DirectoryList.append(alldir)
        
Subdir = next(os.walk(LocalizationMaturesave_dir))

for x in Subdir[1]:
    currentdir = LocalizationMaturesave_dir + '/' + x 
    SubSubdir = next(os.walk(currentdir))
    for y in SubSubdir[1]:
        alldir = LocalizationMaturesave_dir + '/' + x + '/' + y+ '/'
        DirectoryList.append(alldir)
        


In [ ]:
SaveName = 'ONEATBinAV2'
SaveNameVal = 'ONEATBinAV2Validation'


MovieFrames = 7
npzfileGenerator.generate_training_data_V2(DirectoryList, 'DetectionLabel', SaveNpzDirectory, SaveName, SaveNameVal,0, MovieFrames, SizeX, SizeY)
        
SaveName = 'ONEATBinAV2Prediction'
SaveNameVal = 'ONEATBinAV2PredictionValidation'


MovieFrames = 4
npzfileGenerator.generate_training_data_V2(DirectoryList, 'PredictionLabel', SaveNpzDirectory, SaveName, SaveNameVal,0, MovieFrames, SizeX, SizeY)

SaveName = 'ONEATBinAV2PrePrediction'
SaveNameVal = 'ONEATBinAV2PrePredictionValidation'


MovieFrames = 3
npzfileGenerator.generate_training_data_V2(DirectoryList, 'PredictionLabel', SaveNpzDirectory, SaveName, SaveNameVal,0, MovieFrames, SizeX, SizeY)
    

/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neaton/DynamicNormalEventTrainingDataBin2/ResizeOriginal/ResizeOriginal/


100%|██████████| 1068/1068 [00:07<00:00, 134.98it/s]


/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neaton/DynamicNormalEventTrainingDataBin2/ResizeOriginal/Sigma1/


100%|██████████| 1068/1068 [00:07<00:00, 136.40it/s]


/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neaton/DynamicNormalEventTrainingDataBin2/DeformedOriginal/ResizeOriginal/


...

100%|██████████| 11208/11208 [01:38<00:00, 113.86it/s]


/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neaton/DynamicMasterDivisionEventTrainingDataBin2/ShiftLXY/ResizeOriginal/


100%|██████████| 11393/11393 [01:42<00:00, 111.17it/s]


/home/sancere/VarunNewton/CurieTrainingDatasets/Raw_Datasets/Neaton/DynamicMasterDivisionEventTrainingDataBin2/ShiftLX/ResizeOriginal/


 73%|███████▎  | 8249/11335 [01:14<00:32, 95.21it/s] 